In [12]:
import xarray as xr
import numpy as np
import pandas as pd
from scipy.ndimage import convolve1d
from scipy.fft import fft2, ifft2, fftfreq
import matplotlib.pyplot as plt

In [60]:
sample_nc = xr.open_dataset("/Dellwork9/cwubchil/Reanalysis/ERA5/v/v850_2001.nc")

In [11]:
# Merge u/v-winds for S2S reforecast

for year in np.arange(2001,2020+1):
    for month in np.arange(5,9+1):
        for ens in np.arange(1,10+1):
            print(year, month, ens)
            u_nc = xr.open_dataset("/Dellwork9/cwubchil/S2S_Data/ECMWF/u/u_"+str(year)+str(month).zfill(2)+"_ens"+str(ens).zfill(2)+".nc")
            v_nc = xr.open_dataset("/Dellwork9/cwubchil/S2S_Data/ECMWF/v/v_"+str(year)+str(month).zfill(2)+"_ens"+str(ens).zfill(2)+".nc")
            u850 = u_nc["u"].sel({"isobaricInhPa":850})
            v850 = v_nc["v"].sel({"isobaricInhPa":850})
            uv_850 = xr.merge([u850, v850])
            uv_850.to_netcdf("/Dellwork9/cwubchil/S2S_Data/ECMWF/uv/uv850_"+str(year)+str(month).zfill(2)+"_ens"+str(ens).zfill(2)+".nc")
            u_nc.close()
            v_nc.close()

2008 7 4


In [3]:
# Merge u/v-winds for ERA5

for year in np.arange(2001,2020+1):
    u_nc = xr.open_dataset("/Dellwork9/cwubchil/Reanalysis/ERA5/u/u850_"+str(year)+".nc")
    v_nc = xr.open_dataset("/Dellwork9/cwubchil/Reanalysis/ERA5/v/v850_"+str(year)+".nc")
    uv_850 = xr.merge([u_nc, v_nc], compat='override', join='override')
    uv_850.to_netcdf("/Dellwork9/cwubchil/Reanalysis/ERA5/uv/uv850_"+str(year)+".nc")
    u_nc.close()
    v_nc.close()

In [61]:
lat = sample_nc["lat"]
lon = sample_nc["lon"]

In [123]:
# Low-level u/v-winds

v_full = []

for year in np.arange(2001,2020+1):
    year_dates = pd.date_range(str(year)+"-01-01", str(year)+"-12-31")
    v_nc = xr.open_dataset("/Dellwork9/cwubchil/Reanalysis/ERA5/v/v850_"+str(year)+".nc")
    v_full.append(np.array(v_nc["v"]))
    
u_full = []

for year in np.arange(2001,2020+1):
    year_dates = pd.date_range(str(year)+"-01-01", str(year)+"-12-31")
    u_nc = xr.open_dataset("/Dellwork9/cwubchil/Reanalysis/ERA5/u/u850_"+str(year)+".nc")
    u_full.append(np.array(u_nc["u"]))

In [241]:
v_full_arr = np.concatenate(v_full, axis=0)
u_full_arr = np.concatenate(u_full, axis=0)

In [263]:
# Upper-level u/v-winds

v_full_upper = []

for year in np.arange(2001,2020+1):
    year_dates = pd.date_range(str(year)+"-01-01", str(year)+"-12-31")
    #start_dates_ind = np.where(str(year)+"-05-01" == year_dates)[0][0]
    v_upper_nc = xr.open_dataset("/Dellwork9/cwubchil/Reanalysis/ERA5/v/v250_"+str(year)+".nc")
    #v_full.append(np.array(v_nc["v"].sel({"lat": lat_eq})))
    v_full_upper.append(np.array(v_upper_nc["v"]))

u_full_upper = []

for year in np.arange(2001,2020+1):
    year_dates = pd.date_range(str(year)+"-01-01", str(year)+"-12-31")
    #start_dates_ind = np.where(str(year)+"-05-01" == year_dates)[0][0]
    u_upper_nc = xr.open_dataset("/Dellwork9/cwubchil/Reanalysis/ERA5/u/u250_"+str(year)+".nc")
    #v_full.append(np.array(v_nc["v"].sel({"lat": lat_eq})))
    u_full_upper.append(np.array(u_upper_nc["u"]))

v_full_upper_arr = np.concatenate(v_full_upper, axis=0)
u_full_upper_arr = np.concatenate(u_full_upper, axis=0)

In [126]:
k = fftfreq(v_full_arr.shape[2], 1/v_full_arr.shape[2])
w = fftfreq(v_full_arr.shape[0])
P = 1/w
k_mesh, P_mesh = np.meshgrid(k, P)

/tmp/ipykernel_15605/1182738989.py:3: RuntimeWarning: divide by zero encountered in divide
  P = 1/w


In [128]:
# Wavenumber-Frequency Filtering for MRG-TD v-winds

k_lr = (0 <= k_mesh) & (k_mesh <= 14)
P_lr = (2.5 <= P_mesh) & (P_mesh <= 10)
k_ul = (0 >= k_mesh) & (k_mesh >= -14)
P_ul = (-2.5 >= P_mesh) & (P_mesh >= -10)

In [129]:
v_full_arr_detrend = v_full_arr - np.mean(v_full_arr)
v_full_arr_fft = fft2(v_full_arr_detrend, axes=(0,2))
v_full_arr_fft_filtered = np.where(((k_lr & P_lr) | (k_ul & P_ul))[:,None,:], v_full_arr_fft, 0)
v_full_arr_filtered = np.real(ifft2(v_full_arr_fft_filtered, axes=(0,2)))

In [162]:
v_full_arr_filtered_nc = xr.DataArray(data=v_full_arr_filtered, coords={"time": pd.date_range("2001-01-01", "2020-12-31"), "lat":sample_nc["lat"], "lon":sample_nc["lon"]}, name="v_MRG_TD")
v_full_arr_filtered_nc.to_netcdf("v_full_arr_filtered.nc")

In [165]:
# Vorticity

svo_full = []

for year in np.arange(2001,2020+1):
    year_dates = pd.date_range(str(year)+"-01-01", str(year)+"-12-31")
    uv_nc = xr.open_dataset("/Dellwork9/cwubchil/Reanalysis/ERA5/uv/dv850_"+str(year)+".nc")
    svo_full.append(np.array(uv_nc["svo"]))

In [168]:
svo_full_arr = np.concatenate(svo_full, axis=0)

In [184]:
k = fftfreq(svo_full_arr.shape[2], 1/svo_full_arr.shape[2])
w = fftfreq(svo_full_arr.shape[0])
P = 1/w
k_mesh, P_mesh = np.meshgrid(k, P)

/tmp/ipykernel_15605/3989583575.py:3: RuntimeWarning: divide by zero encountered in divide
  P = 1/w


In [186]:
# Wavenumber-Frequency Filtering for ERW Vorticity

k_lr = (1 <= k_mesh) & (k_mesh <= 10)
P_lr = (10 <= P_mesh) & (P_mesh <= 40)
k_ul = (-1 >= k_mesh) & (k_mesh >= -10)
P_ul = (-10 >= P_mesh) & (P_mesh >= -40)

In [181]:
svo_full_arr_detrend = svo_full_arr - np.mean(svo_full_arr)
svo_full_arr_fft = fft2(svo_full_arr_detrend, axes=(0,2))
svo_full_arr_fft_filtered = np.where(((k_lr & P_lr) | (k_ul & P_ul))[:,None,:], svo_full_arr_fft, 0)
svo_full_arr_filtered = np.real(ifft2(svo_full_arr_fft_filtered, axes=(0,2)))

In [199]:
svo_full_arr_filtered_nc = xr.DataArray(data=svo_full_arr_filtered, coords={"time": pd.date_range("2001-01-01", "2020-12-31"), "lat": np.array(uv_nc["latitude"]), "lon": np.array(uv_nc["longitude"])}, name="vort_RW")
svo_full_arr_filtered_nc.to_netcdf("svo_full_arr_filtered.nc")

In [9]:
def low_pass_weights(window, cutoff):
    
    """
    Calculate weights for a low pass Lanczos filter.
    https://github.com/liv0505/Lanczos-Filter/blob/master/lanczosbp.py
    Args:

    window: int
        The length of the filter window.

    cutoff: float
        The cutoff frequency in inverse time steps.
    """

    order = ((window - 1) // 2 ) + 1
    nwts = 2 * order + 1
    w = np.zeros([nwts])
    n = nwts // 2
    w[n] = 2 * cutoff
    k = np.arange(1., n)
    sigma = np.sin(np.pi * k / n) * n / (np.pi * k)
    firstfactor = np.sin(2. * np.pi * cutoff * k) / (np.pi * k)
    w[n-1:0:-1] = firstfactor * sigma
    w[n+1:-1] = firstfactor * sigma
    return w[1:-1]

# Lanczos Filtering of Vorticity and moisture

In [10]:
lfw = low_pass_weights(48, 1/14)
svo_full_arr_lanczos = convolve1d(svo_full_arr, weights=lfw, axis=0, mode="wrap")

In [237]:
svo_full_arr_lanczos_nc = xr.DataArray(data=svo_full_arr_lanczos, coords={"time": pd.date_range("2001-01-01", "2020-12-31"), "lat": np.array(uv_nc["latitude"]), "lon": np.array(uv_nc["longitude"])}, name="vort_lanczos")
svo_full_arr_lanczos_nc.to_netcdf("svo_full_arr_lanczos.nc")

In [242]:
v_full_arr_lanczos = convolve1d(v_full_arr, weights=lfw, axis=0, mode="wrap")
u_full_arr_lanczos = convolve1d(u_full_arr, weights=lfw, axis=0, mode="wrap")

In [267]:
v_full_upper_arr_lanczos = convolve1d(v_full_upper_arr, weights=lfw, axis=0, mode="wrap")
u_full_upper_arr_lanczos = convolve1d(u_full_upper_arr, weights=lfw, axis=0, mode="wrap")

In [269]:
wind_shear_lanczos = np.sqrt((v_full_upper_arr_lanczos - v_full_arr_lanczos)**2 + (u_full_upper_arr_lanczos - u_full_arr_lanczos)**2)

In [273]:
wind_shear_lanczos_nc = xr.DataArray(data=wind_shear_lanczos, coords={"time": pd.date_range("2001-01-01", "2020-12-31"), "lat": np.array(v_nc["lat"]), "lon": np.array(v_nc["lon"])}, name="wind_shear")
wind_shear_lanczos_nc.to_netcdf("wind_shear.nc")

In [4]:
q_full = []

for year in np.arange(2001,2020+1):
    year_dates = pd.date_range(str(year)+"-01-01", str(year)+"-12-31")
    #start_dates_ind = np.where(str(year)+"-05-01" == year_dates)[0][0]
    q_nc = xr.open_dataset("/Dellwork9/cwubchil/Reanalysis/ERA5/q/q700_"+str(year)+".nc")
    #v_full.append(np.array(v_nc["v"].sel({"lat": lat_eq})))
    q_full.append(np.array(q_nc["q"]))

In [6]:
q_full_arr = np.concatenate(q_full, axis=0)

In [11]:
q_full_arr_lanczos = convolve1d(q_full_arr, weights=lfw, axis=0, mode="wrap")

In [15]:
q_lanczos_nc = xr.DataArray(data=q_full_arr_lanczos, coords={"time": pd.date_range("2001-01-01", "2020-12-31"), "lat": np.array(q_nc["lat"]), "lon": np.array(q_nc["lon"])}, name="q")
q_lanczos_nc.to_netcdf("q_lanczos.nc")